In [14]:
from transformers import GPT2Tokenizer
from rusty_dawg import Dawg, PyDawg


# Expects to find a DAWG file in the `dawg` directory below the project root.
dawg_path = "../dawg/wikitext-2-raw.dawg"
dawg = Dawg.load(dawg_path)

# Make sure the tokenizer matches the one used to construct the DAWG.
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [15]:
py_dawg = PyDawg(dawg, tokenizer)

Let's query with a string from Wikitext. We can use `get_suffix_context` to get the suffix contexts as output by the DAWG.

In [16]:
# Substring found in the Wikitext 2 train data.
query = "As with previous Valkyira Chronicles games , Valkyria Chronicles III"

# Get suffix contexts
suffix_contexts = py_dawg.get_suffix_context(query)

print(suffix_contexts)

{'tokens': [1722, 351, 2180, 569, 18354, 8704, 17740, 1830, 837, 569, 18354, 7496, 17740, 6711], 'suffix_contexts': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'context_counts': [234, 9, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


The method `get_matching_substrings` provides a more user-friendly wrapper around `get_suffix_context`. It returns the list of spans of the input query for which matches were found in the training corpus, and gives the `count` indicating the number of times each span appears in the training corpus.

Since this example comes from the training data, we see that the entire query is exactly matched in the training data. 

In [17]:
# Return a list of all substrings in the DAWG that match the query.
matching_substrings = py_dawg.get_matching_substrings(query, remove_redundant=True)

for entry in matching_substrings["matches"]:
    print(entry)

{'tokens': (1722,), 'token_indices': (0,), 'text': 'As', 'count': 234}
{'tokens': (1722, 351), 'token_indices': (0, 1), 'text': 'As with', 'count': 9}
{'tokens': (1722, 351, 2180, 569, 18354, 8704, 17740, 1830, 837, 569, 18354, 7496, 17740, 6711), 'token_indices': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13), 'text': 'As with previous Valkyira Chronicles games, Valkyria Chronicles III', 'count': 1}


Here's a query that isn't included in the training data.

In [18]:
query = "Usain bolt set the world record in the 100-meter dash."
suffix_context = py_dawg.get_suffix_context(query)
matching_substrings = py_dawg.get_matching_substrings(query)

for substring in matching_substrings["matches"]:
    print(substring)  

{'tokens': (5842,), 'token_indices': (0,), 'text': 'Us', 'count': 4}
{'tokens': (391,), 'token_indices': (1,), 'text': 'ain', 'count': 271}
{'tokens': (18100,), 'token_indices': (2,), 'text': ' bolt', 'count': 4}
{'tokens': (900,), 'token_indices': (3,), 'text': ' set', 'count': 725}
{'tokens': (900, 262), 'token_indices': (3, 4), 'text': ' set the', 'count': 29}
{'tokens': (262, 995), 'token_indices': (4, 5), 'text': ' the world', 'count': 347}
{'tokens': (262, 995, 1700), 'token_indices': (4, 5, 6), 'text': ' the world record', 'count': 2}
{'tokens': (995, 1700, 287, 262, 1802), 'token_indices': (5, 6, 7, 8, 9), 'text': ' world record in the 100', 'count': 1}
{'tokens': (12,), 'token_indices': (10,), 'text': '-', 'count': 17027}
{'tokens': (14470,), 'token_indices': (12,), 'text': ' dash', 'count': 7}
{'tokens': (13,), 'token_indices': (13,), 'text': '.', 'count': 8668}
